- 사용 모델 : gpt-4o-mini
- 각 평가 질의에 대해 다양한 표현의 질문 변형

#### 평가셋 랜덤 추출
- eval_queries.json

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from collections import defaultdict, Counter
from datetime import datetime
from pathlib import Path
from typing import List, Dict, Tuple
import numpy as np
import pandas as pd
import json
import os


notebook_dir = Path.cwd()
if notebook_dir.name != 'cache':
    os.chdir(Path(__file__).parent if '__file__' in globals() else Path.cwd())
print(f"현재 작업 디렉토리: {os.getcwd()}")


with open('post_preprocessed_data_v3.json', 'r', encoding='utf-8') as f:
    documents = json.load(f)
category_counts = Counter([d['category_main'] for d in documents if not d['exclude']])
print("\n카테고리 분포:")
for cat, count in category_counts.most_common():
    print(f"  {cat}: {count}")

현재 작업 디렉토리: c:\SKN_19\project\llm\cache

카테고리 분포:
  symptom_diagnosis: 416
  environment_recommendation: 96
  plant_identification: 94
  repotting: 55
  urgent_care: 52
  propagation: 45
  blooming_fruiting: 35
  pruning_shaping: 23
  beginner_faq: 4
  other: 3


In [4]:
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from pinecone import Pinecone
from dotenv import load_dotenv
load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
QNA_INDEX_NAME = "plant-qna-v3"
DIMENSION = 1536  
UPSERT_NAMESPACE = f"{QNA_INDEX_NAME}-openai"

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(QNA_INDEX_NAME)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vector_store = PineconeVectorStore(
    index=index,
    embedding=embeddings,
    namespace=UPSERT_NAMESPACE
)
print(f"VectorStore 초기화 완료 (namespace: {UPSERT_NAMESPACE})")

VectorStore 초기화 완료 (namespace: plant-qna-v3-openai)


In [6]:
# 평가용 질의셋 생성
# 각 카테고리에서 대표적인 쿼리를 추출하여 평가 샘플 생성

import random
random.seed(42)
   
active_docs = [d for d in documents if not d['exclude']]
print(f"활성 문서 수: {len(active_docs)}")

category_docs = defaultdict(list)
for i, doc in enumerate(active_docs):
    category_docs[doc['category_main']].append((i, doc))

# 각 카테고리에서 평가 샘플 추출
eval_samples = []
samples_per_category = {
    'symptom_diagnosis': 10,
    'environment_recommendation': 10,
    'plant_identification': 10,
    'blooming_fruiting': 10,
    'repotting': 10,
    'propagation': 10,
    'urgent_care': 10,
    'pruning_shaping': 10,
    'beginner_faq': 4
}
   
for category, count in samples_per_category.items():
    if category in category_docs:
        available = category_docs[category]
        sample_size = min(count, len(available))
        sampled = random.sample(available, sample_size)
        
        for idx, doc in sampled:
            eval_samples.append({
                'query': doc['question'],
                'gold_ids': [doc['ids']],
                'category_main': category,
                'doc_index': idx,
                'post_id': doc['base_metadata']['post_id'],
                'channel': doc['base_metadata']['channel'],
                'date': doc['base_metadata']['date']
            })

print(f"총 평가 샘플 수: {len(eval_samples)}")
print("\n카테고리별 샘플 분포:")
sample_category_counts = Counter([s['category_main'] for s in eval_samples])
for cat, count in sample_category_counts.most_common():
    print(f"  {cat}: {count}")

# 평가셋 저장
eval_set_path = 'eval_queries.json'
with open(eval_set_path, 'w', encoding='utf-8') as f:
    json.dump(eval_samples, f, ensure_ascii=False, indent=2)


활성 문서 수: 823
총 평가 샘플 수: 84

카테고리별 샘플 분포:
  symptom_diagnosis: 10
  environment_recommendation: 10
  plant_identification: 10
  blooming_fruiting: 10
  repotting: 10
  propagation: 10
  urgent_care: 10
  pruning_shaping: 10
  beginner_faq: 4


#### 질문 변형 평가셋 생성
- augmented_eval_queries_for_rag.json

In [7]:
from collections import defaultdict, Counter
from openai import OpenAI
from tqdm import tqdm
import random
import json
import os


client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

with open('post_preprocessed_data_v3.json', 'r', encoding='utf-8') as f:
    all_docs = json.load(f)

with open('eval_queries.json', 'r', encoding='utf-8') as f:
    eval_samples = json.load(f)

docs_by_id = {doc['ids']: doc for doc in all_docs}
category_docs = defaultdict(list)
for doc in all_docs:
    if not doc.get('exclude', False):
        category_docs[doc['category_main']].append(doc)

print(f"전체 문서 수: {len(all_docs)}")
print(f"평가 샘플 수: {len(eval_samples)}")
print(f"문서 ID 매핑 완료: {len(docs_by_id)}개")

전체 문서 수: 823
평가 샘플 수: 84
문서 ID 매핑 완료: 807개


In [8]:
SYSTEM_PROMPT = """
##당신의 역할:
- 당신은 "식물 상담 Q&A 서비스"의 RAG 평가셋을 고도화하는 데이터 큐레이터입니다.
- 각 기준 문서(base_doc)와 그와 연관될 수 있는 후보 문서들(candidate_docs)을 입력으로 받아,
  1) 질문 변형(query paraphrase)을 여러 개 만들고,
  2) gold_ids를 하나에서 여러 개로 확장하는 일을 합니다.

##목표:
- 현재 평가셋은 `query = 원문 question`, `gold_ids = [해당 문서 1개]`로 구성되어 있습니다.
- 당신은 이를 다음과 같이 더 현실적인 평가셋으로 변환해야 합니다.
  1) 기준 질문과 의미는 같지만 표현이 다른 query를 여러 개 생성
  2) 같은 증상/상황/정보를 다루는 다른 Q&A 문서들도 gold_ids에 포함

##주의사항:
- 답변은 **반드시 JSON 형식**으로만 출력합니다.
- JSON 이외의 설명, 코멘트는 절대 포함하지 마세요.
- Boolean이 아니라 문자열/리스트 위주이므로, null 대신 빈 문자열이나 빈 배열을 사용해도 됩니다.

[해야 할 일 1: 질문 변형 만들기]

1. base_doc.question을 기반으로, 다음 조건을 만족하는 query 변형들을 만듭니다.
   - 의미/의도는 유지하되, 표현/문장 구조/단어 선택을 바꿀 것
   - 실제 사용자가 질문할 법한 자연스러운 한국어 문장일 것
   - 가능한 한 다음과 같은 변화를 섞어서 만들어도 좋습니다.
     - 구어체 ↔ 조금 더 정중한 말투
     - 순서 바꾸기 (증상 → 환경 / 환경 → 증상)
     - "원인에 대한 가설(과습/영양부족/햇빛 등)"을 질문에 포함시키거나 빼보기
     - 단, 증상/식물/핵심 조건 자체는 바꾸지 말 것

2. 생성할 query 개수:
   - 최소 2개, 최대 4개 정도
   - 그 중 1개는 base_doc.question을 그대로 사용해도 됩니다.

[해야 할 일 2: gold_ids 여러 개로 확장]

1. base_doc는 항상 gold_ids에 포함합니다.
   - 즉, 기본적으로 gold_ids에는 `"base_doc.ids"`가 들어가야 합니다.

2. candidate_docs 목록을 모두 검토하고,
   - base_doc의 질문/답변/상황과 **실질적으로 같은 문제를 다루거나,**
   - 증상과 환경이 매우 유사해서 "이 문서들을 함께 참고해도 좋다"고 판단되는 경우,
   - 해당 candidate의 ids를 gold_ids에 추가합니다.

3. gold_ids에 포함될 수 있는 기준 예시:
   - 동일한 식물/유사한 식물 (예: 둘 다 몬스테라, 혹은 둘 다 관엽/비슷한 환경 요구)
   - 증상 패턴이 거의 동일 (예: 잎 끝 갈변, 과습 의심, 겨울철 실내 환경 등)
   - 추천 조치가 유사 (예: 물주기 줄이기, 간접광으로 이동, 통풍 개선 등)

4. gold_ids에 포함되지 않아야 할 예시:
   - 전혀 다른 증상/문제 (예: 하나는 해충, 하나는 냉해)
   - 다른 도메인(예: 환경 추천 vs 식별 질문)만 다루는 문서
   - 식물/환경/증상이 너무 달라서, 이 query에 대한 정답으로 보기 어렵다고 판단되는 문서

5. gold_ids는 중복 없이 배열로 정리합니다.
   - base_doc.ids를 맨 앞에 두고, 나머지 candidate ids를 뒤에 나열합니다.

[출력 형식]

당신은 아래 형식의 JSON 객체 하나를 출력합니다.

{
  "base_id": string,                 // base_doc.ids
  "category_main": string,           // base_doc.category_main
  "base_question": string,           // base_doc.question
  "eval_queries": [
    {
      "query": string,               // 변형된 질문 1개
      "gold_ids": [string, ...]      // base_id + 관련 candidate ids
    },
    {
      "query": string,
      "gold_ids": [string, ...]
    }
    ...
  ]
}

###규칙:
- eval_queries 배열에는 최소 2개, 최대 4개의 항목을 만듭니다.
- 각 eval_queries[*].gold_ids는 모두 같은 리스트여도 괜찮지만,
  - 필요하다면 query의 뉘앙스에 맞춰 조금 다르게 구성해도 됩니다.
- base_id는 항상 gold_ids에 포함되어야 합니다.
"""

In [9]:
def select_candidate_docs(base_doc, all_docs_list, num_candidates=15):
    """
    base_doc와 관련된 candidate 문서들을 선정
    
    선정 기준:
    1. 같은 카테고리 (category_main)
    2. 겹치는 서브카테고리 (category_sub)
    3. base_doc 자신은 제외
    """
    base_id = base_doc['ids']
    base_category = base_doc['category_main']
    base_sub_categories = set(base_doc.get('category_sub', []))
    
    candidates = []
    
    for doc in all_docs_list:
        # base_doc 자신은 제외
        if doc['ids'] == base_id:
            continue
        
        # exclude된 문서 제외
        if doc.get('exclude', False):
            continue
        
        # 점수 계산
        score = 0
        
        # 같은 메인 카테고리면 점수 +10
        if doc['category_main'] == base_category:
            score += 10
        
        # 서브 카테고리 겹치는 개수만큼 점수 추가
        doc_sub_categories = set(doc.get('category_sub', []))
        overlap = base_sub_categories & doc_sub_categories
        score += len(overlap) * 5
        
        if score > 0:
            candidates.append((score, doc))
    
    # 점수 높은 순으로 정렬하고 상위 num_candidates개 선택
    candidates.sort(key=lambda x: x[0], reverse=True)
    selected_candidates = [doc for score, doc in candidates[:num_candidates]]
    
    return selected_candidates

In [10]:
# 질문 변형 평가용 정답 셋 생성 
def call_llm_for_augmentation(base_doc, candidate_docs):
    
    # 입력 데이터 구성
    input_data = {
        "base_doc": {
            "ids": base_doc['ids'],
            "question": base_doc['question'],
            "answer": base_doc['answer'],
            "category_main": base_doc['category_main'],
            "category_sub": base_doc.get('category_sub', []),
            "base_metadata": base_doc.get('base_metadata', {})
        },
        "candidate_docs": [
            {
                "ids": doc['ids'],
                "question": doc['question'],
                "answer": doc['answer'],
                "category_main": doc['category_main'],
                "category_sub": doc.get('category_sub', [])
            }
            for doc in candidate_docs
        ]
    }
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",  
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": json.dumps(input_data, ensure_ascii=False, indent=2)}
            ],
            temperature=0.7,
            response_format={"type": "json_object"}
        )
        
        result = json.loads(response.choices[0].message.content)
        return result
    
    except Exception as e:
        print(f"Error processing {base_doc['ids']}: {str(e)}")
        
        # 에러 발생 시 기본 구조 반환
        return {
            "base_id": base_doc['ids'],
            "category_main": base_doc['category_main'],
            "base_question": base_doc['question'],
            "eval_queries": [
                {
                    "query": base_doc['question'],
                    "gold_ids": [base_doc['ids']]
                }
            ],
            "error": str(e)
        }

In [11]:
def process_eval_samples(eval_samples, all_docs, num_samples=None):
    """
    평가 샘플들을 처리하여 고도화된 평가셋 생성
    
    Args:
        eval_samples: 기존 평가 샘플 리스트
        all_docs: 전체 문서 리스트
        num_samples: 처리할 샘플 수 (None이면 전체)
    """
    augmented_results = []
    
    # 처리할 샘플 수 결정
    samples_to_process = eval_samples[:num_samples] if num_samples else eval_samples
    
    print(f"총 {len(samples_to_process)}개의 평가 샘플 처리 시작...")
    
    for idx, eval_sample in enumerate(tqdm(samples_to_process)):
        # gold_ids의 첫 번째 ID가 base_doc
        base_id = eval_sample['gold_ids'][0]
        
        # base_doc 찾기
        if base_id not in docs_by_id:
            print(f"Warning: {base_id} not found in docs_by_id")
            continue
        
        base_doc = docs_by_id[base_id]
        
        # candidate 문서 선정
        candidate_docs = select_candidate_docs(base_doc, all_docs, num_candidates=15)
        
        # LLM API 호출
        result = call_llm_for_augmentation(base_doc, candidate_docs)
        
        # 메타데이터 추가
        result['original_eval_sample'] = eval_sample
        result['num_candidates_provided'] = len(candidate_docs)
        
        augmented_results.append(result)
        
        # 중간 저장 (10개마다)
        if (idx + 1) % 10 == 0:
            with open('augmented_eval_temp.json', 'w', encoding='utf-8') as f:
                json.dump(augmented_results, f, ensure_ascii=False, indent=2)
    
    print(f"\n처리 완료: {len(augmented_results)}개")
    return augmented_results

In [12]:
def analyze_augmented_eval(augmented_results):

    print("=" * 60)
    print("평가셋 분석")
    print("=" * 60)
    
    total_samples = len(augmented_results)
    total_queries = sum(len(r['eval_queries']) for r in augmented_results)
    
    print(f"\n기본 통계:")
    print(f"  - 총 base 문서 수: {total_samples}")
    print(f"  - 총 생성된 query 변형 수: {total_queries}")
    print(f"  - base 문서당 평균 query 수: {total_queries / total_samples:.2f}")
    
    # gold_ids 개수 분석
    gold_ids_counts = []
    for result in augmented_results:
        for eq in result['eval_queries']:
            gold_ids_counts.append(len(eq['gold_ids']))
    
    print(f"\ngold_ids 확장 통계:")
    print(f"  - 평균 gold_ids 개수: {sum(gold_ids_counts) / len(gold_ids_counts):.2f}")
    print(f"  - 최소 gold_ids 개수: {min(gold_ids_counts)}")
    print(f"  - 최대 gold_ids 개수: {max(gold_ids_counts)}")
    
    # gold_ids 개수별 분포
    from collections import Counter
    gold_ids_distribution = Counter(gold_ids_counts)
    print(f"\ngold_ids 개수 분포:")
    for count in sorted(gold_ids_distribution.keys()):
        print(f"  - {count}개: {gold_ids_distribution[count]}건")
    
    # 카테고리별 분포
    category_counts = Counter([r['category_main'] for r in augmented_results])
    print(f"\n카테고리별 분포:")
    for cat, count in category_counts.most_common():
        print(f"  - {cat}: {count}건")
    
    # 에러 체크
    errors = [r for r in augmented_results if 'error' in r]
    if errors:
        print(f"\n에러 발생: {len(errors)}건")
        for err in errors[:3]:  # 처음 3개만 표시
            print(f"  - {err['base_id']}: {err.get('error', 'Unknown')}")
    else:
        print(f"\n모든 샘플 정상 처리")
    
    return {
        'total_samples': total_samples,
        'total_queries': total_queries,
        'avg_queries_per_sample': total_queries / total_samples,
        'avg_gold_ids': sum(gold_ids_counts) / len(gold_ids_counts),
        'gold_ids_distribution': dict(gold_ids_distribution),
        'category_distribution': dict(category_counts),
        'num_errors': len(errors)
    }


In [13]:
# 결과를 RAG 평가에 사용할 형식으로 변환
def convert_to_eval_format(augmented_results):
    """
    고도화된 결과를 RAG 평가 시스템에서 사용할 형식으로 변환
    
    출력 형식:
    [
      {
        "query": "질문 변형 1",
        "gold_ids": ["groro_3300", "groro_3329"],
        "category_main": "symptom_diagnosis",
        "base_doc_id": "groro_3300",
        "query_variant_idx": 0
      },
      ...
    ]
    """
    eval_format = []
    
    for result in augmented_results:
        base_id = result['base_id']
        category_main = result['category_main']
        
        for idx, eq in enumerate(result['eval_queries']):
            eval_format.append({
                'query': eq['query'],
                'gold_ids': eq['gold_ids'],
                'category_main': category_main,
                'base_doc_id': base_id,
                'query_variant_idx': idx
            })
    
    return eval_format


# 생성된 결과의 샘플(유효성) 검증
def validate_augmented_results(augmented_results, docs_by_id):

    print("=" * 60)
    print("결과 검증")
    print("=" * 60)
    
    issues = []
    
    for idx, result in enumerate(augmented_results):
        base_id = result['base_id']
        
        # 1. base_id가 존재하는지 확인
        if base_id not in docs_by_id:
            issues.append(f"Result {idx}: base_id '{base_id}' not found in docs")
        
        # 2. eval_queries가 비어있지 않은지 확인
        if not result.get('eval_queries'):
            issues.append(f"Result {idx}: eval_queries is empty")
            continue
        
        # 3. 각 query에 대해 검증
        for q_idx, eq in enumerate(result['eval_queries']):
            # query가 비어있지 않은지
            if not eq.get('query') or not eq['query'].strip():
                issues.append(f"Result {idx}, Query {q_idx}: query is empty")
            
            # gold_ids가 비어있지 않은지
            if not eq.get('gold_ids') or len(eq['gold_ids']) == 0:
                issues.append(f"Result {idx}, Query {q_idx}: gold_ids is empty")
                continue
            
            # base_id가 gold_ids에 포함되어 있는지
            if base_id not in eq['gold_ids']:
                issues.append(f"Result {idx}, Query {q_idx}: base_id not in gold_ids")
            
            # 모든 gold_ids가 유효한지 확인
            for gold_id in eq['gold_ids']:
                if gold_id not in docs_by_id:
                    issues.append(f"Result {idx}, Query {q_idx}: gold_id '{gold_id}' not found")
    
    if issues:
        print(f"\n발견된 문제: {len(issues)}건\n")
        for issue in issues[:10]:  # 처음 10개만 표시
            print(f"  - {issue}")
        if len(issues) > 10:
            print(f"  ... 외 {len(issues) - 10}건")
    else:
        print(f"\모든 검증 통과")
    
    return issues

- 실행

In [14]:
# 전체 실행
augmented_eval_full = process_eval_samples(eval_samples, all_docs, num_samples=None)

# 검증
issues = validate_augmented_results(augmented_eval_full, docs_by_id)

# 분석
stats = analyze_augmented_eval(augmented_eval_full)

# 저장 (원본 형식)
with open('augmented_eval_queries_full.json', 'w', encoding='utf-8') as f:
    json.dump(augmented_eval_full, f, ensure_ascii=False, indent=2)

# 평가용 형식으로 변환 및 저장
eval_format = convert_to_eval_format(augmented_eval_full)
with open('augmented_eval_queries_for_rag.json', 'w', encoding='utf-8') as f:
    json.dump(eval_format, f, ensure_ascii=False, indent=2)

print(f"\\n전체 결과가 저장되었습니다:")
print(f"  - 원본 형식: augmented_eval_queries_full.json")
print(f"  - RAG 평가용: augmented_eval_queries_for_rag.json")
print(f"  - 총 평가 쿼리 수: {len(eval_format)}")

총 84개의 평가 샘플 처리 시작...


100%|██████████| 84/84 [13:47<00:00,  9.85s/it]


처리 완료: 84개
결과 검증

발견된 문제: 8건

  - Result 5, Query 1: gold_id 'groro_5764' not found
  - Result 8, Query 3: gold_id 'groro_4401' not found
  - Result 16, Query 0: gold_id 'groro_7360' not found
  - Result 39, Query 0: gold_id 'groro_6540' not found
  - Result 39, Query 1: gold_id 'groro_6540' not found
  - Result 39, Query 2: gold_id 'groro_6540' not found
  - Result 39, Query 3: gold_id 'groro_6540' not found
  - Result 74, Query 3: gold_id 'groro_6836' not found
평가셋 분석

기본 통계:
  - 총 base 문서 수: 84
  - 총 생성된 query 변형 수: 294
  - base 문서당 평균 query 수: 3.50

gold_ids 확장 통계:
  - 평균 gold_ids 개수: 3.24
  - 최소 gold_ids 개수: 1
  - 최대 gold_ids 개수: 4

gold_ids 개수 분포:
  - 1개: 8건
  - 2개: 16건
  - 3개: 168건
  - 4개: 102건

카테고리별 분포:
  - symptom_diagnosis: 10건
  - environment_recommendation: 10건
  - plant_identification: 10건
  - blooming_fruiting: 10건
  - repotting: 10건
  - propagation: 10건
  - urgent_care: 10건
  - pruning_shaping: 10건
  - beginner_faq: 4건

모든 샘플 정상 처리
\n전체 결과가 저장되었습니다:
  - 원본 형식: augmente

#### jhgan/ko-sroberta-multitask

In [ ]:
# from langchain_pinecone import PineconeVectorStore

# # 두 개의 QnA 인덱스 설정
# INDEX_V1_NAME = "plant-qna"     # 기존 인덱스
# INDEX_V3_NAME = "plant-qna-v3"  # 메타데이터 추가 인덱스

# # 각 인덱스에 대한 벡터 스토어 및 retriever 생성
# index_v1 = pc.Index(INDEX_V1_NAME)
# vector_store_v1 = PineconeVectorStore(index=index_v1, embedding=embeddings)
# retriever_v1 = vector_store_v1.as_retriever(search_kwargs={"k": 10})

# index_v3 = pc.Index(INDEX_V3_NAME)
# vector_store_v3 = PineconeVectorStore(index=index_v3, embedding=embeddings)
# retriever_v3 = vector_store_v3.as_retriever(search_kwargs={"k": 10})

# print(f"{INDEX_V1_NAME} retriever 준비 완료")
# print(f"{INDEX_V3_NAME} retriever 준비 완료")


In [ ]:
# # 평가셋 로드
# with open('augmented_eval_queries_for_rag.json', 'r', encoding='utf-8') as f:
#     eval_queries = json.load(f)

# print(f"{len(eval_queries)}개 쿼리")

In [ ]:
# # 평가 함수 정의
# def evaluate_retriever(retriever, eval_queries, k_values=[1, 3, 5, 10]):
#     """
#     Retriever의 성능을 평가합니다.
    
#     Args:
#         retriever: LangChain retriever 객체
#         eval_queries: 평가 쿼리 리스트
#         k_values: 평가할 k 값들
        
#     Returns:
#         results: 각 쿼리별 검색 결과
#         metrics: 평가 지표
#     """
#     results = []
    
#     for query_data in tqdm(eval_queries, desc="검색 수행 중"):
#         query = query_data['query']
#         gold_ids = query_data['gold_ids']
        
#         # 검색 수행
#         retrieved_docs = retriever.invoke(query)
        
#         # 검색된 문서 ID 추출
#         retrieved_ids = []
#         for doc in retrieved_docs:
#             doc_id = doc.metadata.get('id', '')
#             retrieved_ids.append(doc_id)
        
#         results.append({
#             'query': query,
#             'gold_ids': gold_ids,
#             'retrieved_ids': retrieved_ids,
#             'retrieved_docs': retrieved_docs
#         })
    
#     # 평가 지표 계산
#     metrics = calculate_metrics(results, k_values)
    
#     return results, metrics


# def calculate_metrics(results, k_values=[1, 3, 5, 10]):
#     """
#     검색 결과에 대한 평가 지표를 계산합니다.
#     """
#     metrics = {k: {'precision': [], 'recall': [], 'hit_rate': []} for k in k_values}
#     mrr_scores = []
    
#     for result in results:
#         gold_ids = set(result['gold_ids'])
#         retrieved_ids = result['retrieved_ids']
        
#         # MRR 계산
#         for idx, doc_id in enumerate(retrieved_ids, 1):
#             if doc_id in gold_ids:
#                 mrr_scores.append(1.0 / idx)
#                 break
#         else:
#             mrr_scores.append(0.0)
        
#         # k별 지표 계산
#         for k in k_values:
#             retrieved_k = set(retrieved_ids[:k])
            
#             # Precision@k
#             if len(retrieved_k) > 0:
#                 precision = len(retrieved_k & gold_ids) / len(retrieved_k)
#             else:
#                 precision = 0.0
            
#             # Recall@k
#             if len(gold_ids) > 0:
#                 recall = len(retrieved_k & gold_ids) / len(gold_ids)
#             else:
#                 recall = 0.0
            
#             # Hit Rate@k
#             hit = 1.0 if len(retrieved_k & gold_ids) > 0 else 0.0
            
#             metrics[k]['precision'].append(precision)
#             metrics[k]['recall'].append(recall)
#             metrics[k]['hit_rate'].append(hit)
    
#     # 평균 계산
#     summary = {'MRR': np.mean(mrr_scores)}
#     for k in k_values:
#         summary[f'Precision@{k}'] = np.mean(metrics[k]['precision'])
#         summary[f'Recall@{k}'] = np.mean(metrics[k]['recall'])
#         summary[f'HitRate@{k}'] = np.mean(metrics[k]['hit_rate'])
    
#     return summary


In [ ]:
# # plant-qna (기존 인덱스) 평가
# print(f"{'='*50}")
# print(f"{INDEX_V1_NAME} 평가 시작")
# print(f"{'='*50}")

# # results_v1, metrics_v1 = evaluate_retriever(retriever_v1, eval_queries)
# results_v1, metrics_v1 = evaluate_retriever_improved(retriever_v1, eval_queries)

# print("\n평가 완료")
# print("\n=== 평가 결과 ===")
# for metric, value in metrics_v1.items():
#     print(f"{metric}: {value:.4f}")


In [ ]:
# # plant-qna-v3 (메타데이터 추가 인덱스) 평가
# print(f"{'='*50}")
# print(f"{INDEX_V3_NAME} 평가 시작")
# print(f"{'='*50}")

# # results_v3, metrics_v3 = evaluate_retriever(retriever_v3, eval_queries)
# results_v3, metrics_v3 = evaluate_retriever_improved(retriever_v3, eval_queries)

# print("\n평가 완료")
# print("\n=== 평가 결과 ===")
# for metric, value in metrics_v3.items():
#     print(f"{metric}: {value:.4f}")


In [ ]:
# # 결과 비교 데이터프레임 생성
# comparison_df = pd.DataFrame({
#     'Metric': list(metrics_v1.keys()),
#     INDEX_V1_NAME: list(metrics_v1.values()),
#     INDEX_V3_NAME: list(metrics_v3.values())
# })

# # 성능 향상률 계산
# comparison_df['Improvement (%)'] = ((comparison_df[INDEX_V3_NAME] - comparison_df[INDEX_V1_NAME]) 
#                                      / comparison_df[INDEX_V1_NAME] * 100)

# print("\n" + "="*80)
# print("QnA 인덱스 성능 비교")
# print("="*80)
# print(comparison_df.to_string(index=False))
# print("="*80)


---

In [ ]:
# # 임베딩 모델 구현

# from sentence_transformers import SentenceTransformer
# from typing import List
# import torch

# # 1) OpenAI 임베딩 함수 (기존 embeddings 객체 활용)
# def embed_with_openai(texts: List[str], batch_size: int = 100) -> List[List[float]]:
#     """OpenAI text-embedding-3-small 모델로 임베딩 생성"""
#     all_embeddings = []
#     for i in range(0, len(texts), batch_size):
#         batch = texts[i:i + batch_size]
#         batch_embeddings = embeddings.embed_documents(batch)
#         all_embeddings.extend(batch_embeddings)
#     return all_embeddings

# # 2) 한국어 임베딩 함수
# def embed_with_korean_model(texts: List[str], batch_size: int = 32) -> List[List[float]]:
#     """jhgan/ko-sroberta-multitask 모델로 임베딩 생성"""
#     if not hasattr(embed_with_korean_model, 'model'):
#         print("한국어 임베딩 모델 로딩 중... (jhgan/ko-sroberta-multitask)")
#         embed_with_korean_model.model = SentenceTransformer('jhgan/ko-sroberta-multitask')
#         if torch.cuda.is_available():
#             embed_with_korean_model.model = embed_with_korean_model.model.to('cuda')
#             print("  GPU로 실행")
#         else:
#             print("  CPU로 실행")
    
#     model = embed_with_korean_model.model
#     all_embeddings = []
#     for i in range(0, len(texts), batch_size):
#         batch = texts[i:i + batch_size]
#         batch_embeddings = model.encode(batch, convert_to_numpy=True, show_progress_bar=False)
#         all_embeddings.extend(batch_embeddings.tolist())
#     return all_embeddings

# # 3) 임베딩 모델 래퍼 클래스
# class EmbeddingModel:
#     """임베딩 모델 래퍼 클래스"""
#     def __init__(self, model_name: str):
#         self.model_name = model_name
#         self.dimension = 1536 if model_name == 'openai' else 768
        
#     def embed(self, texts: List[str]) -> List[List[float]]:
#         if self.model_name == 'openai':
#             return embed_with_openai(texts)
#         elif self.model_name == 'sroberta':
#             return embed_with_korean_model(texts)
#         else:
#             raise ValueError(f"Unknown model: {self.model_name}")
    
#     def embed_query(self, query: str) -> List[float]:
#         if self.model_name == 'openai':
#             return embeddings.embed_query(query)
#         elif self.model_name == 'sroberta':
#             if not hasattr(embed_with_korean_model, 'model'):
#                 embed_with_korean_model([query])
#             return embed_with_korean_model.model.encode([query], convert_to_numpy=True)[0].tolist()
#         else:
#             raise ValueError(f"Unknown model: {self.model_name}")

# print("임베딩 모델 함수 구현 완료")
# print("  - OpenAI text-embedding-3-small (dimension: 1536)")
# print("  - jhgan/ko-sroberta-multitask (dimension: 768)")


임베딩 모델 함수 구현 완료
  - OpenAI text-embedding-3-small (dimension: 1536)
  - jhgan/ko-sroberta-multitask (dimension: 768)


In [ ]:
# # 한국어 모델 인덱스 생성 및 데이터 적재

# from pinecone import ServerlessSpec

# print("=" * 80)
# print("한국어 모델 인덱스 생성 및 데이터 적재")
# print("=" * 80)

# # 한국어 모델용 새 인덱스 (dimension이 768로 다르므로 별도 인덱스 필요)
# KOREAN_INDEX_NAME = "plant-qna-v2"
# KOREAN_DIMENSION = 768
# KOREAN_NAMESPACE = f"{KOREAN_INDEX_NAME}-sroberta"

# print(f"\n적재 대상 문서 수: {len(docs_to_upsert)}개")

# # 한국어 모델용 인덱스 생성/로드
# if KOREAN_INDEX_NAME not in [idx["name"] for idx in pc.list_indexes()]:
#     print(f"\n'{KOREAN_INDEX_NAME}' 인덱스 생성 중...")
#     pc.create_index(
#         name=KOREAN_INDEX_NAME,
#         dimension=KOREAN_DIMENSION,
#         metric='cosine',
#         spec=ServerlessSpec(cloud="aws", region="us-east-1")
#     )
#     print(f"'{KOREAN_INDEX_NAME}' 인덱스 생성 완료")
# else:
#     print(f"\n'{KOREAN_INDEX_NAME}' 인덱스 이미 존재")

# korean_index = pc.Index(KOREAN_INDEX_NAME)
# print(f"인덱스 연결 완료")

# # 기존 데이터 확인
# existing_stats = korean_index.describe_index_stats()
# print(f"\n현재 인덱스 상태:")
# print(f"  - 총 벡터 수: {existing_stats.get('total_vector_count', 0)}")
# print(f"  - 네임스페이스: {list(existing_stats.get('namespaces', {}).keys())}")

# # 사용자 확인
# print(f"\n적재 설정:")
# print(f"  - 인덱스: {KOREAN_INDEX_NAME}")
# print(f"  - 네임스페이스: {KOREAN_NAMESPACE}")
# print(f"  - Dimension: {KOREAN_DIMENSION}")
# print(f"  - 문서 수: {len(docs_to_upsert)}개")

# response = input("\n한국어 모델 임베딩을 생성하고 적재하시겠습니까? (y/n): ")

# if response.lower() == 'y':
#     print("\n한국어 임베딩 생성 시작...")
    
#     texts_to_embed = []
#     for doc in docs_to_upsert:
#         text = f"Question: {doc['question']}\nAnswer: {doc['answer']}"
#         texts_to_embed.append(text)
    
#     korean_model = EmbeddingModel('sroberta')
#     vectors = korean_model.embed(texts_to_embed)
#     print(f"\n임베딩 완료: {len(vectors)}개")
    
#     insert_data = []
#     for doc, vector in zip(docs_to_upsert, vectors):
#         metadata = {
#             'ids': doc['ids'],
#             'text': f"Question: {doc['question']}\nAnswer: {doc['answer']}",
#             'question': doc['question'],
#             'answer': doc['answer'][:500],
#             'category_main': doc['category_main'],
#             'category_sub': doc.get('category_sub', []),
#             'post_id': doc['base_metadata']['post_id'],
#             'channel': doc['base_metadata']['channel'],
#             'date': doc['base_metadata']['date'],
#             'source_platform': doc['base_metadata']['source_platform']
#         }
#         insert_data.append({'id': doc['ids'], 'values': vector, 'metadata': metadata})
    
#     def batch(iterable, n=100):
#         for idx in range(0, len(iterable), n):
#             yield iterable[idx: idx + n]
    
#     print(f"\nPinecone에 배치 업로드 중... (namespace: {KOREAN_NAMESPACE})")
#     for idx, batch_data in enumerate(batch(insert_data, 100)):
#         korean_index.upsert(vectors=batch_data, namespace=KOREAN_NAMESPACE)
#         print(f"  Batch {idx+1}/{(len(insert_data)-1)//100 + 1} 완료 ({len(batch_data)}개)")
    
#     print(f"\n총 {len(insert_data)}개 문서 적재 완료!")
#     print(f"   네임스페이스: {KOREAN_NAMESPACE}")
    
#     updated_stats = korean_index.describe_index_stats()
#     print(f"\n업데이트된 인덱스 상태:")
#     print(f"  - 총 벡터 수: {updated_stats.get('total_vector_count', 0)}")
#     for ns, info in updated_stats.get('namespaces', {}).items():
#         print(f"  - {ns}: {info.get('vector_count', 0)}개")
# else:
#     print("\n 적재를 건너뜁니다.")

# print("\n" + "=" * 80)
